# 安装Opencv

从配置文件创建虚拟环境并且激活

```bash
conda create -f envs.yaml
conda activate py_opencv
```

在 `notebook` 中选择 `python` 解释器，点一下顶上右边的选择解释器，也就是图中的 `py_opencv(Python 3.13.5)` 部分


![notebook head](images\notebookhead.png)


在弹出的菜单中选择 `Python环境` 

![notebook head](images\py_select_1.png)


在弹出的菜单中选择 `py_opencv` 环境

![notebook head](images\py_select_2.png)


# 计算机眼中的图片

## 图像的本质——像素、坐标与颜色

### 像素 (Pixel) - 图像的基本单元

“Pixel”是“Picture Element”（图像元素）的缩写。它是构成数字图像的最小、最基本的单位。你可以把它想象成一个无法再被分割的纯色小方块。我们所看到的任何一张高清、细节丰富的数码照片，放大、放大、再放大，最终看到的都会是这些纯色的小方块。

![pixels show](./images/weixin_screenshot.png)

这个图片里面的坐标就是当前鼠标对应的像素的坐标

**一张图像的核心属性，就是由所有像素的属性共同决定的。**

每个像素，都有两个核心信息：

1. 它的位置：它在这张马赛克拼图的哪个位置？
2. 它的值：它应该显示什么颜色？

其实还有一种图是`矢量图`，这种图没有像素的概念，类似于使用数学公式描述画面。所以其不受图像放大的影响，清晰度不会变。但是在导入opencv的进行处理的时候，都会转为一般的像素图片，所以我们这里不再多说。

### 坐标系 (Coordinate System) - 像素的位置

为了管理千千万万个像素的位置，计算机使用了一个二维坐标系。请注意，这个坐标系和我们初中数学学的直角坐标系有一个关键区别：

1. 原点 (0, 0) 在图像的左上角。
2. X 轴 水平向右延伸，代表图像的宽度 (Width)。
3. Y 轴 垂直向下延伸，代表图像的高度 (Height)。

举个例子，对于一张分辨率为 800 x 600 (宽x高) 的图片：

1. 左上角第一个像素的坐标是 (x=0, y=0)。
2. 右上角最后一个像素的坐标是 (x=799, y=0)。
3. 右下角最后一个像素的坐标是 (x=799, y=599)。
4. 图像正中心的像素坐标大约是 (x=400, y=300)。

但是，存储的时候又会反过来，所以在使用的时候，注意一下就可以了。

### 颜色与通道 (Color & Channels) - 定义像素的值

我们已经知道了像素的位置，那如何定义它的颜色呢？这就是“通道”概念的用武之地。

#### 最简单的情况：灰度图 (Grayscale Image)

一张黑白照片就是一张灰度图。它只有一个颜色通道。

+ 每个像素的值是一个单独的数字，用来表示其亮度。
+ 这个数字的范围通常是 0 到 255 (这被称为8位图像，因为 2^8 = 256)。
+ 0 代表纯黑色，255 代表纯白色，中间的数值则代表不同程度的灰色。

![pixels show](./images/grey_sample.png)

#### 彩色图像 (Color Image)

我们日常见到的大多数图片都是彩色的。计算机通过混合不同强度的三原色光来创造出成千上万种颜色。因此，一张彩色图通常有**三个颜色通道**。

+ 最常见的颜色模型是 RGB (Red, Green, Blue)。
+ ⚠️ 重要陷阱： OpenCV 默认使用的颜色通道顺序是 BGR (Blue, Green, Red)！ 这和很多其他软件（如Photoshop, Matplotlib）的RGB顺序不同。在进行颜色相关的操作时，一定要牢记这一点，否则你会得到奇怪的颜色。
+ 在BGR模型中，每个像素的值不再是一个数字，而是由 三个数字组成的元组 (Tuple) 或列表，分别代表该像素中蓝色、绿色和红色的强度。
+ 每个值的范围同样是 0 到 255。

#### 带有透明度的彩色图 (Four-Channel Image)

一张四通道图，通常被称为 **BGRA** 或 **RGBA** 图。

+ 前三个字母我们已经很熟悉了：B (Blue), G (Green), R (Red)。它们决定了像素的颜色。
+ 新增的第四个字母 A 代表 Alpha 通道。

**Alpha通道不控制颜色，而是控制像素的“不透明度 (Opacity)”或“透明度 (Transparency)”。**

和颜色通道一样，Alpha通道的值通常也是一个 0 到 255 的数字：

+ Alpha = 255 (完全不透明)：这是最“实心”的状态。像素会完全显示它自己的颜色，遮挡住它后面的一切。一张普通的JPG照片里所有像素的Alpha值都可以看作是255。
+ Alpha = 0 (完全透明)：像素是“隐形”的。它不显示任何颜色，像一块完全干净的玻璃，会直接显示出它后面的背景。
+ 0 < Alpha < 255 (半透明)：像素处于半透明状态，像一块有颜色的玻璃或一个水印。它会显示自己的颜色，但同时也会透出背景的颜色。Alpha的值越低，像素就越透明。

#### RGB-D图像-携带深度信息

**RGB-D** 代表 **Red, Green, Blue + Depth**。它不是一张单一的图像，而是一组同步并对齐的数据，通常包含两个部分：

+ 一张RGB（或BGR）图像： 这就是我们已经熟悉的标准彩色图像。它告诉我们场景中每个像素的颜色是什么。
+ 一张深度图 (Depth Map)： 这是新增的关键信息。它是一张单通道的“图像”，但它的像素值不是颜色或亮度，而是距离。

![](./images/RGBD-sample.jpg)

### 总结

**图像就是一个巨大的数字矩阵**

我们把所有知识点串联起来：

一张位图是一个由像素构成的网格（由坐标系定义），网格上每个点都有一个或多个数值来代表其颜色（由通道定义）。

这种 **“网格结构 + 数值”** 的组合，在数学和编程中有一个完美对应的概念——**矩阵 (Matrix)**，或者叫 **多维数组 (Multi-dimensional Array)**。

+ 一张灰度图，可以看作一个 二维矩阵。矩阵的形状是 (height, width)。矩阵中的每个元素就是该像素位置的灰度值（0-255）。
+ 一张彩色图，可以看作一个 三维矩阵。矩阵的形状是 (height, width, 3)。这里的 "3" 就代表B, G, R三个通道。你可以把它想象成3个二维矩阵叠在一起。



# OpenCV

> OpenCV 的全称是 Open Source Computer Vision Library，翻译过来就是“开源计算机视觉库”。

## 读取储存图片

### 读取图像

```python
image = cv2.imread(filepath, flags)
```
- `filepath`：图像文件的路径。
- `flags`: 一个可选参数，用于指定图片的读取方式。
    - cv2.IMREAD_COLOR (默认值): 以三通道BGR彩色图像格式读取。即使原始图片有透明度（Alpha通道），也会被忽略。
    - cv2.IMREAD_GRAYSCALE: 以单通道灰度图像格式读取。
    - cv2.IMREAD_UNCHANGED: 读取所有通道，包括Alpha通道（如果存在）。对于PNG等格式，这将返回一个四通道BGRA图像。

### 保存图像

```python
cv2.imwrite(filepath, image)
```

- filepath: 你希望保存的文件名和路径。文件的扩展名至关重要，例如：
    - '.jpg' 或 '.jpeg': 会以有损压缩的JPEG格式保存。适用于照片。
    - '.png': 会以无损压缩的PNG格式保存。适用于需要保留精确细节或透明度的图像。
- image: 你想要保存的图像矩阵。

### 显示图像

显示图像需要三个函数协同工作：`cv2.imshow()`, `cv2.waitKey()`, `cv2.destroyAllWindows()`。

#### `cv2.imshow(window_name, image)`
*   **功能**: 在一个窗口中显示图像。如果窗口不存在，则自动创建。
*   `window_name` (`str`): 窗口的标题和唯一标识符。
*   `image`: 要显示的图像矩阵。

#### `cv2.waitKey(delay)`
*   **功能**: 暂停程序并等待键盘输入，这是让窗口持续显示的关键。
*   `delay` (`int`): 等待的毫秒数。
    *   **`delay = 0`**: **无限等待**，直到用户按下任意键。用于显示单张图片。
    *   **`delay > 0`**: 等待指定毫秒数。用于播放视频流，例如 `cv2.waitKey(1)`。
*   **返回值**: 返回被按下的键的ASCII码，如果没有按键则返回`-1`。

#### `cv2.destroyAllWindows()`
*   **功能**: 关闭所有由OpenCV创建的窗口，释放资源。

In [1]:
import cv2

# --- 1. 读取 ---
img_path = '.\images\RGBD-sample.jpg'
image = cv2.imread(img_path)

# --- 2. 检查 ---
if image is None:
    print(f"错误: 无法在路径 '{img_path}' 找到图片。")
else:
    # --- 3. 显示 ---
    window_title = 'RobotMaster Demo'
    cv2.imshow(window_title, image)
    print(f"图片已在窗口 '{window_title}' 中显示。按任意键关闭。")
    
    # 等待用户交互
    cv2.waitKey(0)
    
    # 销毁窗口
    cv2.destroyAllWindows()
    print("窗口已关闭。")

    # --- 4. 处理与写入 ---
    # 将图像转为灰度
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 保存灰度图像
    output_path = '.\images\RGBD-sample-gray.png'
    cv2.imwrite(output_path, gray_image)
    print(f"处理后的图像已保存到 '{output_path}'。")

<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:28: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:28: SyntaxWarning: invalid escape sequence '\i'
C:\Users\18384\AppData\Local\Temp\ipykernel_31832\3759751766.py:4: SyntaxWarning: invalid escape sequence '\i'
  img_path = '.\images\RGBD-sample.jpg'
C:\Users\18384\AppData\Local\Temp\ipykernel_31832\3759751766.py:28: SyntaxWarning: invalid escape sequence '\i'
  output_path = '.\images\RGBD-sample-gray.png'


图片已在窗口 'RobotMaster Demo' 中显示。按任意键关闭。
窗口已关闭。
处理后的图像已保存到 '.\images\RGBD-sample-gray.png'。


### 视频流

视频是一系列连续的图像帧。处理视频的核心是一个循环，在循环中对每一帧进行读取和处理。

#### 读取视频 (`cv2.VideoCapture`)

创建一个“视频捕捉对象”来从数据源获取视频帧。

**语法**
```python
cap = cv2.VideoCapture(source)
```
*   `source`: 数据源。
    *   **摄像头**: 整数 `0` (默认), `1`, `2`, ...
    *   **视频文件**: 字符串路径，如 `'my_video.mp4'`。

**常用方法**
*   `cap.isOpened()`: 检查视频源是否成功打开。
*   `ret, frame = cap.read()`: 读取下一帧。`ret` 为 `True` 表示成功，`frame` 是图像。
*   `cap.release()`: 释放视频源。

#### 写入视频 (`cv2.VideoWriter`)

创建一个“视频写入对象”来将一帧帧图像保存成视频文件。

**语法**
```python
writer = cv2.VideoWriter(filepath, fourcc, fps, frame_size)
```
**参数**
*   `filepath` (`str`): 输出文件名 (如 `'output.avi'`)。
*   `fourcc`: 指定视频编码器的4字节代码。常用 `cv2.VideoWriter_fourcc(*'XVID')`。
*   `fps` (`float`): 输出视频的帧率（每秒帧数）。
*   `frame_size` (`tuple`): `(width, height)`，视频分辨率。

**常用方法**
*   `writer.write(frame)`: 写入一帧。
*   `writer.release()`: 完成写入并释放资源。

#### 示例：从摄像头录制视频并保存

In [2]:
import cv2

# --- 1. 创建读取和写入对象 ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("错误: 无法打开摄像头。")
    exit()

# 获取视频帧的尺寸
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (w, h)

# 定义编码器和创建VideoWriter对象
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter('output.avi', fourcc, 20.0, size)

print("正在录制... 按 'q' 键停止。")

# --- 2. 循环处理 ---
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 将帧写入文件
    writer.write(frame)
    
    # 显示实时画面
    cv2.imshow('Recording...', frame)
    
    # 等待1ms，并检查退出键
    if cv2.waitKey(1) == ord('q'):
        break

# --- 3. 释放资源 ---
print("停止录制，正在保存文件...")
cap.release()
writer.release()
cv2.destroyAllWindows()
print("程序结束。")

正在录制... 按 'q' 键停止。
停止录制，正在保存文件...
程序结束。


## 图像处理

### 图像预处理 Image Preprocessing

#### 颜色空间转换 (`cv2.cvtColor`)

+ 目的： 简化计算，方便处理

+ 核心转换：
    + BGR → 灰度 (Grayscale): cv2.COLOR_BGR2GRAY，大多数分析算法的基础。

#### 图像平滑与滤波 (Image Smoothing & Filtering)

+ 目的：去除图像中的噪点（例如，传感器噪声），防止其干扰后续处理。
+ 核心方法：
    + 高斯模糊 (cv2.GaussianBlur): 最常用的滤波方式，通过加权平均来平滑图像。
+ 关键参数：核大小 (Kernel Size) 的概念及其对模糊程度的影响。

In [5]:
# 导入OpenCV库
import cv2

# --- 步骤 1: 加载原始图像 ---
# 我们将使用这张图片作为所有预处理操作的起点。
print("正在加载原始图像...")
image_bgr = cv2.imread('./images/xiaogong.jpg')

# 健壮性检查：确保图像已成功加载
if image_bgr is None:
    print("错误：无法加载！请检查文件路径。")
    exit() # 如果图片没加载，后续操作无意义，直接退出程序

print("图像加载成功！")


# --- 步骤 2: 颜色空间转换 (`cv2.cvtColor`) ---
# 这是预处理中最常见的一步。

# 2.1 BGR -> 灰度 (Grayscale)
# 目的：将三通道的彩色信息压缩为单通道的亮度信息。
#       这可以大大减少计算量，并帮助算法专注于形状和纹理，而不是颜色。
print("正在将图像转换为灰度图...")
image_gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)


# --- 步骤 3: 图像平滑/模糊 (`cv2.GaussianBlur`) ---
# 目的：减少图像中的细节和噪声，这有助于防止后续的边缘检测或阈值处理
#       对无关紧要的噪点产生错误响应。

# 3.1 对原始彩色图进行高斯模糊
# (5, 5)是高斯核的大小，它必须是正奇数。核越大，图像就越模糊。
# 第三个参数是sigmaX，设为0表示由OpenCV根据核大小自动计算。
print("正在对原始彩色图进行高斯模糊...")
blurred_bgr = cv2.GaussianBlur(image_bgr, (5, 5), 0)


# 3.2 对灰度图进行高斯模糊 (这在实际应用中更常见)
print("正在对灰度图进行高斯模糊...")
blurred_gray = cv2.GaussianBlur(image_gray, (9, 9), 0) # 使用一个更大的核来观察效果


# --- 步骤 4: 显示所有处理结果进行对比 ---
# 使用cv2.imshow()将所有处理阶段的图像并排显示，以便直观地理解每个操作的效果。
print("正在显示所有结果图像...")

# 显示原始图像和其模糊版本
cv2.imshow('1. Original BGR', image_bgr)
cv2.imshow('2. Blurred BGR (5x5)', blurred_bgr)

# 显示颜色空间转换的结果
cv2.imshow('3. Grayscale', image_gray)

# 显示灰度图和其模糊版本
cv2.imshow('5. Blurred Grayscale (9x9)', blurred_gray)


# --- 步骤 5: 等待用户交互并清理 ---
print("\n所有窗口已显示。按任意键关闭所有窗口并退出程序。")
cv2.waitKey(0)  # 无限等待用户按键
cv2.destroyAllWindows() # 关闭所有打开的窗口
print("程序已结束。")

正在加载原始图像...
图像加载成功！
正在将图像转换为灰度图...
正在对原始彩色图进行高斯模糊...
正在对灰度图进行高斯模糊...
正在显示所有结果图像...

所有窗口已显示。按任意键关闭所有窗口并退出程序。
程序已结束。


> ### **滤波原理详解：图像平滑的艺术**
> 
> #### **1. 核心思想：像素的“商量”与“妥协”**
> 
> 想象一张图片里有一个“噪点”——一个像素因为传感器错误，颜色值与周围格格不入（比如在一片黑色中突然出现一个纯白色的像素点）。这个噪点非常“突兀”，非常“尖锐”。
> 
> **滤波（或称平滑、模糊）的核心思想非常朴素：不要让一个像素“一意孤行”，它的最终颜色应该由它和它的“邻居们”一起“商量”决定。**
> 
> 通过取一个像素和它周围邻域像素值的**平均值**或**加权平均值**来替代该像素的原始值，就可以让那个突兀的噪点“妥协”，使其颜色值更接近于周围的像素，从而达到平滑图像、去除噪声的效果。
> 
> 这个“商量”的过程，在数学上被称为 **卷积 (Convolution)**。
> 
> #### **2. 关键工具：核 (Kernel)**
> 
> 为了实现“商量”，我们需要一个“规则模板”，这个模板决定了：
> 1.  “商量”的范围有多大？（是和周围一圈8个邻居商量，还是和两圈24个邻居商量？）
> 2.  每个邻居的“发言权”有多重？（是所有邻居都同等重要，还是离得近的邻居更重要？）
> 
> 这个“规则模板”，就叫做 **核 (Kernel)** 或 **卷积核 (Convolution Kernel)**。它本质上是一个**微型矩阵**。
> 
> #### **3. 工作流程：卷积 (Convolution) 的“滑动窗口”**
> 
> 卷积的过程就像一个“滑动窗口”在图像上移动。我们用一个简单的例子——**均值滤波 (Box Blur)**——来解释这个流程。
> 
> **假设我们有一个3x3的均值滤波核：**
> ```
>      1  1  1
> K = (1/9) * [ 1  1  1 ]
>      1  1  1
> ```
> 这个核的意思是：取3x3邻域内所有像素的值，求和，再除以9（取平均值）。所有邻居的“发言权”都是一样的。
> 
> **工作步骤：**
> 1.  **选择一个目标像素**：假设我们要计算图像中 `P5` 点的新像素值。
> 
> 2.  **对齐核的中心**：将核的中心（Anchor Point）与目标像素 `P5` 对齐。
> 
> 3.  **覆盖邻域**：此时，核会覆盖住 `P5` 和它周围的8个邻居，形成一个3x3的区域。
>     ```
>     原始图像像素值:          卷积核 K:
>     [ P1  P2  P3 ]         [ 1  1  1 ]
>     [ P4  P5  P6 ]         [ 1  1  1 ]
>     [ P7  P8  P9 ]         [ 1  1  1 ]
>     ```
> 
> 4.  **加权求和**：将图像邻域内的每个像素值与核中对应位置的权重相乘，然后将所有结果相加。
>     `Sum = (P1*1) + (P2*1) + ... + (P9*1)`
> 
> 5.  **计算新值**：将总和除以核的权重总和（这里是9）。
>     `New_P5_Value = Sum / 9`
> 
> 6.  **赋值**：用这个计算出的新值，来更新目标像素 `P5` 的值。
> 
> 7.  **滑动窗口**：将核向右移动一个像素，对齐新的目标像素，重复步骤2-6，直到遍历完图像中的所有像素。
> 
> 这个过程，就完成了对整张图像的一次均值滤波。
> 
> #### **4. 升级版：高斯滤波 (`cv2.GaussianBlur`)**
> 
> 均值滤波虽然简单，但有个缺点：它认为邻域内所有像素的“发言权”都一样，这显得有些“粗暴”。直觉上，**距离中心像素越近的邻居，应该有越大的发言权**。
> 
> **高斯滤波 (Gaussian Blur)** 就采纳了这个更合理的思想。它的核不是简单的全1矩阵，而是根据**高斯函数（正态分布/“钟形曲线”）**生成的一个权重矩阵。
> 
> **一个3x3的高斯核可能长这样：**
> ```
>       1  2  1
> K = (1/16) * [ 2  4  2 ]
>       1  2  1
> ```
> 
> *   **中心权重最高 (4)**：目标像素自身的权重最大。
> *   **距离越远，权重越低**：紧邻的像素权重为2，对角线的像素权重只有1。
> *   **权重总和**：`1+2+1+2+4+2+1+2+1 = 16`，所以最后需要除以16。
> 
> **高斯滤波的效果：**
> *   它产生的模糊效果更平滑、更自然，更好地保留了边缘信息，是实际应用中最常用的滤波方法。
> 
> #### **回到OpenCV的代码**
> 
> 现在，我们再来看`cv2.GaussianBlur`这个函数就豁然开朗了：
> 
> `cv2.GaussianBlur(image, (ksize_x, ksize_y), 0)`
> 
> *   `image`: 输入的图像矩阵。
> *   `(ksize_x, ksize_y)`: 这就是我们指定的**核的大小 (Kernel Size)**！例如 `(5, 5)`。
>     *   **核越大**，意味着“商量”的邻居范围越广，**图像就会越模糊**。
>     *   这就是为什么 `(9, 9)` 的模糊效果比 `(5, 5)` 更强烈。
> *   `0`: 这是标准差 `sigmaX`。设为0时，OpenCV会根据你提供的核大小自动计算出一个最合理的高斯分布。
> 
> **总结**
> 
> | 概念 | 解释 |
> | :--- | :--- |
> | **滤波/平滑** | 通过像素与邻居的“商量”来去除噪声、平滑图像。 |
> | **卷积** | 实现“商量”的数学过程，即“滑动窗口”式的加权求和。 |
> | **核 (Kernel)** | “商量”的规则模板，定义了邻域范围和每个邻居的发言权重。 |
> | **均值滤波** | 最简单的滤波，所有邻居权重相同。 |
> | **高斯滤波** | 更高级的滤波，遵循“离中心越近，权重越高”的原则，效果更自然。 |